# An Analysis of Paleo to present Climate Change Data

<details>

<summary> Student Details </summary>

| Student| Details |
| -------- | -------- |
| Course: | KDATG_L08_Y1 |
| Author: | Rebecca Hannah Quinn |
| Student Number: | G00425671 |

</details>

---

## Project Goals

To Analyse CO2 vs Temperature Anomaly for the past 800,000 years to the present day.
To examine the change in the polar-ice coverage alongside this.
To examine the changes in Ireland and Irish climate change signals.

In gathering this information and data I will fuse and analyse the data using pandas dataframes and export the results to csv and json formats.

I will be anaylising the data, the trends, the relationships between them including but not limited to: temporal leads, lags, and frequencies. I will also use synthesised data to predict global temperature anomoly over the next few decades and compare to published models of the same to see if atmospherice CO2 trends continue and comment on the accelerated warming based on the latest features with temperatures polar-ice coverage and ocean and sea levels.




<details>

<summary> Table of Contents </summary>

[PRE-PROCESSING](#01)

[SECTION 2](#02)

[SECTION 3](#03)

[SECTION 4](#04)

[SECTION 5](#05)

[SECTION 6](#06)

[References](#07)

</details>

---


## Introduction


There have been studies on the Earth"s climate that have resulted in data as far back as 800,000 years by extracting core samples from deep beneath the ice sheets of Greenland and Antarctica. These samples contain detailed information on air temperature and CO2 levels that are trapped within them. Current polar records demonstrate a close association between atmospheric carbon dioxide and temperature in the natural world. In essence, when one increases, the other one follows.

However, there is still some uncertainty about which occurred first - a spike in temperature or CO2. Until now, the most extensive records to date on a significant change in Earth"s climate came from the EPICA Dome C ice core on the Antarctic Plateau. The data, which covered the end of the last ice age, between 20,000 and 10,000 years ago, indicated that CO2 levels could have lagged behind rising global temperatures by as much as 1,400 years.
[1]: https://www.scientificamerican.com/article/ice-core-data-help-solve/#:~:text=Scientists%20use%20air%20trapped%20in,than%20the%20ice%20surrounding%20them.


<a id="01i">

## Pre-processing

</a>





### Import Packages

In [45]:
#importing packages required for analysis and visualization
import pandas as pd
import openpyxl 

### Initial Adjustments

#### Reading in the Data

#### Datafile One

Read in data and rename columns to make clearer and pull data from the specific excel sheet for use in analysis later. Both datasets have information rows before the data starts so they are removed during the read in of the data.

In [46]:
epica1col = ["year", "co2", "sigma"]
epica8kyr1 = pd.read_excel("https://www.ncei.noaa.gov/pub/data/paleo/icecore/antarctica/antarctica2015co2.xls", sheet_name="CO2 Composite", skiprows = range(1, 15), names= epica1col)
epica8kyr1.to_csv("epica8kyr1.csv")

#### Datafile Two

Read in data and rename columns to make clearer.

In [47]:
epica2col = ["date", "co2", "unc"]
epica8kyr2 = pd.read_csv("https://gml.noaa.gov/webdata/ccgg/trends/co2/co2_annmean_mlo.csv", skiprows=44, sep=",", names= epica2col)
epica8kyr2.to_csv("epica8kyr2.csv")

#### Cleanup of Data

Here we take the "year" column in the second dataset and convert the data to actual dates (stored in a new column titled "date") and put in order and make clearer so when we merge the datasets we can do so seamlessly.

### Addition of missing Data

Data from 1900 to 1950 missing between the two Datafiles above

In [49]:

###


---

<a id="02i">

## SECTION 2

</a>

In [50]:
#SECTION 2 - PYTHON CELL



---

<a id="03i">

## SECTION 3

</a>

In [51]:
#SECTION 3 - PYTHON CELL



---

<a id="04i">

## SECTION 4

</a>



In [52]:
#SECTION 4 - PYTHON CELL



---

<a id="05i">

## SECTION 5

</a>



In [53]:
#SECTION 5 - PYTHON CELL



---

<a id="06i">

## SECTION 6

</a>



In [54]:
#SECTION 6 - PYTHON CELL



---

<a id="07i">

## REFERENCES

</a>


---